## Exercícios

> Retirados de [learn-python: sqlalchemy_orm-questions](https://aviadr1.github.io/learn-advanced-python/11_db_access/exercise/sqlalchemy_orm-questions.html).

#### Q1.

Baixa e extraia o arquivo compactado com o banco de dados [Chinook database](https://www.sqlitetutorial.net/sqlite-sample-database/). Salve o arquivo `chinook.db` na mesma pasta deste script.
* Link para baixar:


<img width=500 src=https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg>


#### Q2.

Leia o código e os comentários das células a seguir para entender como acessamos os modelos ORM de um banco já existente.

In [1]:
from sqlalchemy import create_engine, text, MetaData, select, func, desc
from sqlalchemy.orm import Session, join

engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

### extrai as classes da base de dados Chinook
metadata = MetaData()
metadata.reflect(engine)

# O metadata tem informações sobre as tabelas
# que serão usadas para criar os modelos ORM
for table_name, table in metadata.tables.items():
    print(table_name)
    print(table.columns.keys())
    print(table.columns.items())
    print('-'*25)

### configura o objeto Base mapeando os modelos ORM das tabelas
from sqlalchemy.ext.automap import automap_base
Base = automap_base(metadata=metadata)
Base.prepare()

# o objeto Base tem os modelos ORM que podemos usar
# para manipular o banco de dados
print(Base.classes.items())

albums
['AlbumId', 'Title', 'ArtistId']
[('AlbumId', Column('AlbumId', INTEGER(), table=<albums>, primary_key=True, nullable=False)), ('Title', Column('Title', NVARCHAR(length=160), table=<albums>, nullable=False)), ('ArtistId', Column('ArtistId', INTEGER(), ForeignKey('artists.ArtistId'), table=<albums>, nullable=False))]
-------------------------
artists
['ArtistId', 'Name']
[('ArtistId', Column('ArtistId', INTEGER(), table=<artists>, primary_key=True, nullable=False)), ('Name', Column('Name', NVARCHAR(length=120), table=<artists>))]
-------------------------
customers
['CustomerId', 'FirstName', 'LastName', 'Company', 'Address', 'City', 'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email', 'SupportRepId']
[('CustomerId', Column('CustomerId', INTEGER(), table=<customers>, primary_key=True, nullable=False)), ('FirstName', Column('FirstName', NVARCHAR(length=40), table=<customers>, nullable=False)), ('LastName', Column('LastName', NVARCHAR(length=20), table=<customers>, nullable=F

In [2]:
# A seguir um exemplo de query na tabela Albums
# usamos o objeto Base para acessar cada modelo ORM.

session = Session(engine)
res = session.scalars(select(Base.classes.albums))
first_album = res.first()
print(first_album.AlbumId, "-", first_album.Title)

1 - For Those About To Rock We Salute You


#### Q3. 
Com base nos códigos anteriores realize as operações solicitadas nas células a seguir:


In [3]:
### Imprima os três primeiros registros da tabela tracks

tracks = Base.classes.tracks
invoice_items = Base.classes.invoice_items
albums = Base.classes.albums
artists = Base.classes.artists

res = session.scalars(select(tracks)).fetchmany(3)

for track in res:
    data = [
        f"\nID: {track.TrackId}",
        f"Album: {track.albums.Title}",
        f"Name: {track.Name}",
        f"Composer: {track.Composer}",
        f"Millisecond: {track.Milliseconds}",
        f"Bytes: {track.Bytes}"
    ]
    
    for i in data:
        print(i)


ID: 1
Album: For Those About To Rock We Salute You
Name: For Those About To Rock (We Salute You)
Composer: Angus Young, Malcolm Young, Brian Johnson
Millisecond: 343719
Bytes: 11170334

ID: 2
Album: Balls to the Wall
Name: Balls to the Wall
Composer: None
Millisecond: 342562
Bytes: 5510424

ID: 3
Album: Restless and Wild
Name: Fast As a Shark
Composer: F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman
Millisecond: 230619
Bytes: 3990994


In [4]:
### Imprima o nome da faixa e o título do álbum das primeiras 20 faixas na tabela tracks.

res = session.scalars(select(tracks)).fetchmany(20)

print("Track name ------ Album title\n")
for track in res:
    print(track.Name, "------", track.albums.Title)


Track name ------ Album title

For Those About To Rock (We Salute You) ------ For Those About To Rock We Salute You
Balls to the Wall ------ Balls to the Wall
Fast As a Shark ------ Restless and Wild
Restless and Wild ------ Restless and Wild
Princess of the Dawn ------ Restless and Wild
Put The Finger On You ------ For Those About To Rock We Salute You
Let's Get It Up ------ For Those About To Rock We Salute You
Inject The Venom ------ For Those About To Rock We Salute You
Snowballed ------ For Those About To Rock We Salute You
Evil Walks ------ For Those About To Rock We Salute You
C.O.D. ------ For Those About To Rock We Salute You
Breaking The Rules ------ For Those About To Rock We Salute You
Night Of The Long Knives ------ For Those About To Rock We Salute You
Spellbound ------ For Those About To Rock We Salute You
Go Down ------ Let There Be Rock
Dog Eat Dog ------ Let There Be Rock
Let There Be Rock ------ Let There Be Rock
Bad Boy Boogie ------ Let There Be Rock
Problem Child 

In [5]:
### Imprima as 10 primeiras vendas de faixas da tabela invoice_items
### Para essas 10 primeiras vendas, imprima os nomes das faixas vendidas e a quantidade vendida.

res = session.scalars(select(invoice_items)).fetchmany(10)

for item in res:
    print(item.tracks.Name, "--->", item.Quantity)

Balls to the Wall ---> 1
Restless and Wild ---> 1
Put The Finger On You ---> 1
Inject The Venom ---> 1
Evil Walks ---> 1
Breaking The Rules ---> 1
Dog Eat Dog ---> 1
Overdose ---> 1
Love In An Elevator ---> 1
Janie's Got A Gun ---> 1


In [6]:
### Imprima os nomes das 10 faixas mais vendidas e quantas vezes foram vendidas.

query = select(invoice_items, func.count(invoice_items.TrackId)).group_by(invoice_items.TrackId).order_by(func.count(invoice_items.TrackId).desc())
# query = text("SELECT TrackId, SUM(Quantity) AS total FROM invoice_items GROUP BY TrackId ORDER BY total DESC")
    

res = session.execute(query).fetchmany(10)

for row in res:
    print(row[0].tracks.Name, "--->", row[1])


Balls to the Wall ---> 2
Inject The Venom ---> 2
Snowballed ---> 2
Overdose ---> 2
Deuces Are Wild ---> 2
Not The Doctor ---> 2
Por Causa De Você ---> 2
Welcome Home (Sanitarium) ---> 2
Snowblind ---> 2
Cornucopia ---> 2


In [7]:
### Quem são os 10 artistas que mais venderam?
### dica: você precisa juntar as tabelas invoice_items, tracks, albums e artists

# Construir e executar a query
query = (
    session.query(
        artists.Name.label("Artista"),
        func.sum(invoice_items.UnitPrice * invoice_items.Quantity).label("TotalVendas")
    )
    .join(albums, albums.ArtistId == artists.ArtistId)
    .join(tracks, tracks.AlbumId == albums.AlbumId)
    .join(invoice_items, invoice_items.TrackId == tracks.TrackId)
    .group_by(artists.ArtistId)
    .order_by(func.sum(invoice_items.UnitPrice * invoice_items.Quantity).desc())
    .limit(10)
)

# Mostrar resultados
for artista, total in query:
    print(f"{artista}: ${total:.2f}")

Iron Maiden: $138.60
U2: $105.93
Metallica: $90.09
Led Zeppelin: $86.13
Lost: $81.59
The Office: $49.75
Os Paralamas Do Sucesso: $44.55
Deep Purple: $43.56
Faith No More: $41.58
Eric Clapton: $39.60
